In [528]:
import yfinance as yf
import pandas as pd
from pandas_datareader.data import DataReader
from datetime import datetime
import requests
import requests_cache
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import exchange_calendars as xcals
from bs4 import BeautifulSoup
import ujson
import numpy as np

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/yuntiangu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [380]:
xses = xcals.get_calendar('XSES')
last_trading_day = xses.date_to_session('2022-04-03', direction='previous').strftime("%Y-%m-%d")
last_trading_day
xses.is_session(datetime.today().strftime("%Y-%m-%d"))

False

In [303]:
portfolio_tickers = [ (str(s)[str(s).find('(') + 1 : str(s).find(')')].split(':'))[-1] + '.SI' for s in list(pd.read_html('https://sginvestors.io/analysts/sti-straits-times-index-constituents-target-price')[0]['Straits Times Index STI Constituent'])]

In [339]:
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite')
# just add headers to your session and provide it to the reader
session.headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
                   'Accept': 'application/json;charset=utf-8'}

In [340]:
portfolio_tickers.remove('na.SI')

ValueError: list.remove(x): x not in list

In [323]:
dff = yf.download(' '.join(portfolio_tickers))

[*********************100%***********************]  30 of 30 completed


In [310]:
yf.Tickers(portfolio_tickers).info

AttributeError: 'Tickers' object has no attribute 'info'

In [330]:
dff.to_csv('./yf_data_downloaded.csv')

In [379]:
for ticker in portfolio_tickers:
    DataReader(ticker, 'yahoo', end=last_trading_day, session=session).to_csv(f'./yf_data/{last_trading_day}_{ticker}.csv')

In [ ]:
pd.read_html('https://sginvestors.io/analysts/sti-straits-times-index-constituents-target-price')[0]

In [ ]:
yf.tick

In [ ]:
testList = portfolio_tickers[:5]
testList

In [ ]:
for ticker in testList:
    print(f'{ticker} News:', yf.Ticker(ticker))
    break

In [ ]:
u11 = yf.Ticker('U11.SI')

In [427]:
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite')
# just add headers to your session and provide it to the reader
session.headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
                   'Accept': 'application/json;charset=utf-8'}



In [432]:
def initialise_portfolio_info_data(port_ticks):
    all_info = []
    for ticker in port_ticks:
        sInfo = yf.Ticker(ticker).info
        all_info.append(sInfo)

    return pd.DataFrame(all_info)
    

In [433]:
info_df = initialise_portfolio_info_data(portfolio_tickers)
info_df.head()

,zip,sector,fullTimeEmployees,longBusinessSummary,city,phone,country,companyOfficers,website,maxAge,...,bidSize,dayHigh,regularMarketPrice,preMarketPrice,logo_url,trailingPegRatio,underlyingSymbol,underlyingExchangeSymbol,headSymbol,uuid
0,048624,Financial Services,24346.0,"United Overseas Bank Limited, together with it...",Singapore,65 6533 9898,Singapore,[],https://www.uobgroup.com,1,...,0.0,32.12,31.96,None,https://logo.clearbit.com/uobgroup.com,0.9106,NaN,NaN,NaN,NaN
1,018982,Financial Services,33000.0,DBS Group Holdings Ltd provides financial prod...,Singapore,65 6878 8888,Singapore,[],https://www.dbs.com,1,...,0.0,35.73,35.55,None,https://logo.clearbit.com/dbs.com,1.2243,NaN,NaN,NaN,NaN
2,579701,Industrials,23187.0,"ComfortDelGro Corporation Limited, an investme...",Singapore,65 6383 8833,Singapore,[],https://www.comfortdelgro.com,1,...,0.0,1.50,1.49,None,https://logo.clearbit.com/comfortdelgro.com,NaN,NaN,NaN,NaN,NaN
3,098632,Industrials,16393.0,"Keppel Corporation Limited, an investment hold...",Singapore,65 6270 6666,Singapore,[],https://www.kepcorp.com,1,...,0.0,6.46,6.46,None,https://logo.clearbit.com/kepcorp.com,3.7558,NaN,NaN,NaN,NaN
4,569873,Technology,12000.0,Venture Corporation Limited provides technolog...,Singapore,65 6482 1755,Singapore,[],https://www.venture.com.sg,1,...,0.0,17.63,17.62,None,https://logo.clearbit.com/venture.com.sg,NaN,NaN,NaN,NaN,NaN


In [463]:
name_sym = info_df[['longName', 'symbol']].copy()
relLoc = name_sym.loc[name_sym['longName']== 'United Overseas Bank Limited']
# relLoc['symbol']
name_sym['longNameCaps'] = [x.upper() for x in list(name_sym['longName'])]
name_sym

,longName,symbol,longNameCaps
0,United Overseas Bank Limited,U11.SI,UNITED OVERSEAS BANK LIMITED
1,DBS Group Holdings Ltd,D05.SI,DBS GROUP HOLDINGS LTD
2,ComfortDelGro Corporation Limited,C52.SI,COMFORTDELGRO CORPORATION LIMITED
3,Keppel Corporation Limited,BN4.SI,KEPPEL CORPORATION LIMITED
4,Venture Corporation Limited,V03.SI,VENTURE CORPORATION LIMITED
5,CapitaLand Integrated Commercial Trust,C38U.SI,CAPITALAND INTEGRATED COMMERCIAL TRUST
6,Ascendas Real Estate Investment Trust,A17U.SI,ASCENDAS REAL ESTATE INVESTMENT TRUST
7,Singapore Telecommunications Limited,Z74.SI,SINGAPORE TELECOMMUNICATIONS LIMITED
8,Oversea-Chinese Banking Corporation Limited,O39.SI,OVERSEA-CHINESE BANKING CORPORATION LIMITED
9,Mapletree Commercial Trust,N2IU.SI,MAPLETREE COMMERCIAL TRUST


In [531]:
def get_sentimental_analysis(info_df):
    name_sym = info_df[['longName', 'symbol']].copy()
    name_sym['longNameCaps'] = [x.upper() for x in list(name_sym['longName'])]
    name_sym['sentiment'] = ujson.dumps({})
    relevant_atags = []
    
    res = session.get('https://sginvestors.io/news/company-announcement/latest/')
    soup = BeautifulSoup(res.text, 'html.parser')
    
    all_ann = soup.find_all("div", class_="corpannouncementitem list-group-item")
    all_corp_ann = soup.select_one("#corporate-announcements")
    a_tags_corp_ann = all_corp_ann.find_all('a')
    
    today = datetime.today().strftime("%Y-%m-%d")
    for tag in a_tags_corp_ann:
#         print(tag.find('div', class_="corpann_info").getText().split(' ')[2])
        if tag.find('span', class_='corpann_stock').getText() in list(name_sym['longNameCaps']):
#             if tag.find('div', class_="corpann_info").getText().split(' ')[2] == today: 
            relevant_atags.append(tag)
    
    print('==========relevant_tags========', relevant_atags)
    
    
    relevant_sentimental_analysis = []
    for relevant_tag in relevant_atags:
        corp_name = relevant_tag.find('span', class_='corpann_stock').getText()
        desc = relevant_tag.find('span', class_="corpann_descr").getText()
#         print('desc', desc)
        lyzer = SentimentIntensityAnalyzer()
        tag_sentiment = lyzer.polarity_scores(desc)
#         print(ujson.dumps(tag_sentiment))
#         name_sym.loc[name_sym['longNameCaps'] == corp_name]['sentiment'] = ujson.dumps(tag_sentiment)
        #['sentiment'] = lyzer.polarity_scores(desc)
        name_sym['sentiment'] = np.where(name_sym['longNameCaps'] == corp_name, ujson.dumps(tag_sentiment), name_sym['sentiment'])
    
    
    return name_sym

In [533]:
get_sentimental_analysis(info_df).to_csv('./test_sentimental_dailyData.csv')

==========relevant_tags======== [<a href="https://links.sgx.com/1.0.0/corporate-announcements/OYROP2BOJK7EPAGP/a93bbadc1bf2c166ed212dc4876dd3c755b319149bcbb47096bac87e8867f870" rel="nofollow" target="corpannoucement"><div class="corpannouncementitem list-group-item" id="corpannouncementitem-364088"><div class="corpann_title"><span class="corpann_stock">KEPPEL CORPORATION LIMITED</span> - <span class="corpann_descr">Incorporation of wholly- owned subsidiaries</span></div><div class="corpann_info"><span><i aria-hidden="true" class="fa fa-clock-o"></i>  2022-04-01 22:19:00</span>   <span><i aria-hidden="true" class="fa fa-bullhorn"></i>  Asset Acquisitions and Disposals</span></div></div></a>]


In [273]:
res.text
soup = BeautifulSoup(res.text, 'html.parser')
all_ann = soup.find_all("div", class_="corpannouncementitem list-group-item")

In [447]:
# all_corp_ann = soup.find("div", {"id": "corporate-announcements"})
all_corp_ann = soup.select_one("#corporate-announcements")
a_tags_corp_ann = all_corp_ann.find_all('a')
print(a_tags_corp_ann[0].find('div', class_="corpann_info").getText().split(' ')[2])
# all_corp_ann_sgx_links = all_corp_ann.find_all('a')
# all_corp_ann_sgx_links[0].find('i', {"class": "fa fa-clock-o"}).getText()

2022-04-02


In [286]:
desc = a_tags_corp_ann[1].find('span', class_="corpann_descr").getText()

In [292]:
a_tags_corp_ann[1].find('span', class_='corpann_stock').getText()

'KTL GLOBAL LIMITED'

In [ ]:
print(soup.prettify())

In [478]:
import ujson
testD = {
    'hi': "hi"
}
ujson.dumps(testD)

'{"hi":"hi"}'

In [283]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/yuntiangu/nltk_data...


True

In [287]:
lyzer = SentimentIntensityAnalyzer()
lyzer.polarity_scores(desc)

{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound': -0.296}